In [4]:
import ortools
from ortools.init.python import init
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np

# 0. Data import and solver set up

In [5]:
distances = pd.read_excel('distances.xlsx', index_col=0, skiprows=1, usecols=range(1, 6))
print("Shape of distances dataset : ",distances.shape)
display(distances.head())
index_values = pd.read_excel('indexValues.xlsx', header=None, names=["indexValue"], usecols=[1])
print("Shape of indexValues dataset : ",index_values.shape)
display(index_values.head())
x_origin = pd.read_excel('x_origin.xlsx', index_col=0,skiprows=1, usecols=range(1, 6))
print("Shape of x' dataset : ",x_origin.shape)
display(x_origin.head())

Shape of distances dataset :  (22, 4)


,1,2,3,4
1,16.16,24.08,24.32,21.12
2,19.00,26.47,27.24,17.33
3,25.29,32.49,33.42,12.25
4,0.00,7.93,8.31,36.12
5,3.07,6.44,7.56,37.37


Shape of indexValues dataset :  (22, 1)


,indexValue
0,0.1609
1,0.1164
2,0.1026
3,0.1516
4,0.0939


Shape of x' dataset :  (22, 4)


,1,2,3,4
x'ij,,,,
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,1,0,0,0
5,1,0,0,0


In [6]:
# Initialize solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# 1. Variables

In [7]:
num_bricks = len(index_values)
num_sr = len(distances.columns)
penalty_constant = 10 

# Decision variables
# assignment[i][j] = 1 if brick i is assigned to SR j
assignment = [[solver.BoolVar(f'assignment_{i}_{j}') for j in range(num_sr)] for i in range(num_bricks)]

print("Number of brics : ", num_bricks)
print("Number of SR : ", num_sr)

Number of brics :  22
Number of SR :  4


# 2. Constraints

In [8]:
# 1. Each brick is assigned to exactly one SR
for i in range(num_bricks):
    solver.Add(sum(assignment[i][j] for j in range(num_sr)) == 1)

# 2. Weight constraints for each SR (total weight between 0.8 and 1.2)
for j in range(num_sr):
    total_weight = solver.Sum(assignment[i][j] * index_values.iloc[i,0] for i in range(num_bricks))
    solver.Add(total_weight >= 0.8)
    solver.Add(total_weight <= 1.2)

# 3. Objective functions

In [9]:
# Objective function: minimize weighted distance + weight-based disruption penalty

objective = solver.Objective()
for i in range(num_bricks):
    for j in range(num_sr):
        # Weighted distance cost
        distance_cost = distances.iloc[i,j] * index_values.iloc[i,0]
        
        # Disruption penalty if reassigned to a different SR
        if (i + 1) in x_origin and x_origin.iloc[i + 1,0] != j + 1:
            disruption_penalty = penalty_constant * index_values.iloc[i,0]
        else:
            disruption_penalty = 0

        # Total cost for assigning brick i to SR j
        total_cost = distance_cost + disruption_penalty
        objective.SetCoefficient(assignment[i][j], total_cost)

objective.SetMinimization()

# 4. Solve

In [10]:
# Solve the problem
status = solver.Solve()

# Print results
if status == pywraplp.Solver.OPTIMAL:
    print('Solution found:')
    # Initialize lists to store weights and distances per SR
    sr_weights = [0.0] * num_sr
    sr_distances = [0.0] * num_sr

    for j in range(num_sr):
        sr_bricks = []
        for i in range(num_bricks):
            if assignment[i][j].solution_value() > 0.5:
                sr_weights[j] += index_values.iloc[i,0]
                sr_distances[j] += distances.iloc[i,j]
                sr_bricks.append(i)
        print(f"SR {j+1} total weight is {round(sr_weights[j],2)} / total distances is {round(sr_distances[j],2)} / assigned bricks are {sr_bricks}")
    # Calculate the sum of (sum of weights per SR * sum of distances per SR)
    total_weighted_distance = sum(sr_weights[j] * sr_distances[j] for j in range(num_sr))
    
    print('Sum of weighted distance:', total_weighted_distance)
    print('Optimal Objective Value:', objective.Value())
else:
    print('No optimal solution found.')

Solution found:
SR 1 total weight is 1.02 / total distances is 35.92 / assigned bricks are [3, 4, 5, 6, 7, 8, 18, 20]
SR 2 total weight is 1.04 / total distances is 7.53 / assigned bricks are [10, 12, 13, 17]
SR 3 total weight is 1.11 / total distances is 6.57 / assigned bricks are [9, 14, 15, 16]
SR 4 total weight is 0.82 / total distances is 113.77 / assigned bricks are [0, 1, 2, 11, 19, 21]
Sum of weighted distance: 144.785327
Optimal Objective Value: 23.123659999999997
